In [1]:
pip install pyshark


   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.8 MB 621.2 kB/s eta 0:00:06
   ----- ---------------------------------- 0.5/3.8 MB 621.2 kB/s eta 0:00:06
   -------- ------------------------------- 0.8/3.8 MB 621.2 kB/s eta 0:00:05
   -------- ------------------------------- 0.8/3.8 MB 621.2 kB/s eta 0:00:05
   ---------- ----------------------------- 1.0/3.8 MB 621.2 kB/s eta 0:00:05
   ---------- ----------------------------- 1.0/3.8 MB 621.2 kB/s eta 0:00:05
   ------------- -------------------------- 1.3/3.8 MB 627.1 kB/s eta 0:00:04
   ------------- -------------------------- 1.3/3.8 MB 627.1 kB/s eta 0:00:04
   ---------------- ----------------------- 1.6/3.8 MB 621.2 kB/s eta 0:00:04
   ---------------- --

In [8]:
pip install scapy



   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 409.0 kB/s eta 0:00:05
   -------- ------------------------------- 0.5/2.4 MB 409.0 kB/s eta 0:00:05
   -------- ------------------------------- 0.5/2.4 MB 409.0 kB/s eta 0:00:05
   ------------ --------------------------- 0.8/2.4 MB 381.3 kB/s eta 0:00:05
   ------------ --------------------------- 0.8/2.4 MB 381.3 kB/s eta 0:00:05
   ------------ --------------------------- 0.8/2.4 MB 381.3 kB/s eta 0:00:05
   ------------ --------------------------- 0.8/2.4 MB 381.3 kB/s eta 0:00:05
   ------------ -----------------------

In [22]:
import pyshark

def parse_layers(file_path):
    # Tạo một FileCapture để đọc file
    cap = pyshark.FileCapture(file_path, display_filter='eth || ip || ipv6')

    print(" Phân tích các gói tin tầng 2 và tầng 3 trong file .pcapng:\n")

    for i, pkt in enumerate(cap[:10]):  # chỉ phân tích 10 gói đầu để dễ xem
        print(f" Gói tin {i + 1}:")

        # Tầng 2: Ethernet
        if 'eth' in pkt:
            eth = pkt.eth
            print("   Layer 2 - Ethernet:")
            print(f"    MAC nguồn: {eth.src}")
            print(f"    MAC đích:  {eth.dst}")
            print(f"    Ether Type: {eth.type}")

        # Tầng 3: IPv4
        if 'ip' in pkt:
            ip = pkt.ip
            print("   Layer 3 - IPv4:")
            print(f"    IP nguồn: {ip.src}")
            print(f"    IP đích:  {ip.dst}")
            print(f"    Giao thức: {ip.proto}")

        # Tầng 3: IPv6
        elif 'ipv6' in pkt:
            ipv6 = pkt.ipv6
            print("   Layer 3 - IPv6:")
            print(f"    IP nguồn: {ipv6.src}")
            print(f"    IP đích:  {ipv6.dst}")
            print(f"    Next Header: {ipv6.nxt}")

        print("-" * 40)

    cap.close()

#  Gọi hàm với file bạn đã tải lên
parse_layers("tcp(1).pcapng")
 

 Phân tích các gói tin tầng 2 và tầng 3 trong file .pcapng:



TypeError: '>=' not supported between instances of 'slice' and 'int'

In [ ]:
from scapy.all import rdpcap, Ether, IP, IPv6

def parse_pcap_with_scapy(file_path):
    packets = rdpcap(file_path)

    for i, packet in enumerate(packets[:10]):  # Hiển thị 10 gói đầu
        print(f"\n📦 Gói tin {i + 1}:")

        # Tầng 2: Ethernet
        if Ether in packet:
            eth = packet[Ether]
            print(f"  🧷 Layer 2 - Ethernet:")
            print(f"    MAC nguồn: {eth.src}")
            print(f"    MAC đích:  {eth.dst}")
            print(f"    Ether Type: {hex(eth.type)}")

        # Tầng 3: IP hoặc IPv6
        if IP in packet:
            ip = packet[IP]
            print(f"  🌐 Layer 3 - IPv4:")
            print(f"    IP nguồn: {ip.src}")
            print(f"    IP đích:  {ip.dst}")
            print(f"    Giao thức: {ip.proto}")

        elif IPv6 in packet:
            ipv6 = packet[IPv6]
            print(f"   Layer 3 - IPv6:")
            print(f"    IP nguồn: {ipv6.src}")
            print(f"    IP đích:  {ipv6.dst}")
            print(f"    Next Header: {ipv6.nh}")

# 📂 Gọi hàm với file bạn tải lên
parse_pcap_with_scapy(r"tcp.pcapng")



Scapy_Exception: Not a supported capture file

In [9]:
import subprocess

def extract_layer2_layer3_info(file_path):
    # Lệnh tshark: trích xuất thông tin từ Ethernet và IP
    cmd = [
        "tshark",
        "-r", file_path,
        "-T", "fields",
        "-e", "frame.number",
        "-e", "eth.src",
        "-e", "eth.dst",
        "-e", "ip.src",
        "-e", "ip.dst",
        "-e", "ip.proto",
        "-E", "separator=|"
    ]

    try:
        output = subprocess.check_output(cmd, text=True)

        print(" Kết quả từ file PCAPNG:\n")
        lines = output.strip().split('\n')
        for line in lines[:10]:  # Chỉ in 10 gói đầu
            fields = line.split('|')
            print(f" Frame {fields[0]}:")
            print(f"   MAC nguồn: {fields[1]}")
            print(f"   MAC đích:  {fields[2]}")
            print(f"   IP nguồn:  {fields[3]}")
            print(f"   IP đích:   {fields[4]}")
            print(f"    Protocol:  {fields[5]}")
            print()

    except FileNotFoundError:
        print(" Lỗi: Bạn cần cài đặt tshark (Wireshark CLI) để dùng lệnh này.")
    except subprocess.CalledProcessError as e:
        print(" Lỗi khi chạy tshark:", e.output)

# Gọi hàm với file đã tải lên
extract_layer2_layer3_info("tcp.pcapng")


 Lỗi: Bạn cần cài đặt tshark (Wireshark CLI) để dùng lệnh này.


In [10]:
import pyshark
# Đường dẫn đến file .pcapng đã thu được bằng Wireshark
path = r'tcp.pcapng' #Chú ý: Ghi đúng đường dẫn trên máy 
 # của Anh/Chị
# Tạo đối tượng đọc file gói tin, chỉ lọc các gói HTTP Request
cap = pyshark.FileCapture(path, display_filter='http.request')
print("Phân tích gói HTTP REQUEST chứa từ khoá 'login' hoặc 
'test'\n")
# Duyệt qua từng gói tin trong file
for i, pkt in enumerate(cap):
try:
# Chuyển toàn bộ nội dung HTTP sang dạng chữ thường để tìm kiếm dễ hơn
http_info = str(pkt.http).lower()
# Nếu trong nội dung gói có chứa 'login' hoặc 'test'
if 'login' in http_info or 'test' in http_info:
print("="*50)
print(f"GÓI #{i+1} Có chứa từ khoá")
# Hiển thị thời gian bắt được gói tin
print("Thời gian:", pkt.sniff_time)
# Hiển thị IP nguồn (máy gửi request) và IP đích (máy chủ)
print("IP nguồn:", pkt.ip.src if hasattr(pkt, 'ip') 
else 'N/A')
print("IP đích:", pkt.ip.dst if hasattr(pkt, 'ip')
else 'N/A')
# Hiển thị phương thức HTTP: GET hoặc POST
if hasattr(pkt.http, 'request_method'):
print("Phương thức:", pkt.http.request_method)
# Hiển thị URL đầy đủ nếu có
if hasattr(pkt.http, 'host') and hasattr(pkt.http, 
'request_uri'):
print("URL:", 
f"http://{pkt.http.host}{pkt.http.request_uri}")
# Hiển thị Cookie nếu gói có gửi cookie
if hasattr(pkt.http, 'cookie'):
print("Cookie:", pkt.http.cookie)
# Hiển thị dữ liệu gửi đi trong phần thân (POST form data)
if hasattr(pkt.http, 'file_data'):
print("Payload:", pkt.http.file_data)
# Nếu có lỗi (ví dụ gói tin không có lớp http), thì tiếp tục gói sau
except Exception as e:
print(f"[Lỗi tại gói #{i+1}]: {e}")

SyntaxError: unterminated string literal (detected at line 7) (722501342.py, line 7)